In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,019 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,003 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,026 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/ma

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-27 07:57:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2023-03-27 07:57:52 (10.1 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [7]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://data-183-proj4-uwa.s3.ap-southeast-2.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)
    
# Show dataframe
df.show()    

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [8]:
from pyspark.sql.functions import isnan, when, count, col

# Check for duplicates
df.select(count("*")).distinct().show()

+--------+
|count(1)|
+--------+
|  960872|
+--------+



In [9]:
# Check for column names
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [10]:
# Check for missing values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|          0|          0|        0|         0|             0|            0|               0|          0|            0|          0|   0|                0|              1|        191|         44|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [11]:
# Get the number of rows in the DataFrame.
df.count()

960872

In [12]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# Transform the Data

## Create the "review_id_table".

In [13]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

# Select the desired columns

review_id_df = df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))

# Cast "customer_id" and "product_parent" column to integer
review_id_df = review_id_df.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))

# Show the resulting DataFrame
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892| 2015-08-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407| 2015-08-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857| 2015-08-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     685450910| 2015-08

## Create the "products" Table

In [14]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EVX7V1I|Game Time Women's...|
|B009S4DODY|XOXO Women's XO55...|
|B00LBKXQRW|Anne Klein Women'...|
|B0009P679Y|Invicta Men's 993...|
|B00DHF30RU|M&c Women's | Cla...|
|B00NIDA43Y|GuTe Classic Skel...|
|B008EQDDPQ|Nautica Men's N13...|
|B004VRBZ66|Timex Men's T2N63...|
|B009BEO81I|        Fossil Riley|
|B008B39MTI|XOXO Women's XO55...|
|B00TGPM8PU|Handmade Wooden W...|
|B00VNXQQQ0|Eterna 2520-41-64...|
|B00B1PV1C4|Nautica Men's N19...|
|B00N1Y8TQ4|Tissot Men's T095...|
|B00G6DBTY6|red line Men's RL...|
|B00HM04AYI|Columbia Men's Fi...|
|B00VI8HB96|GUESS I90176L1 Wo...|
|B00IT25WJU|LanTac DGN556B Dr...|
|B0106S12XE|Skmei S Shock Ana...|
|B00FPSJ63Y|Michael Kors Ladi...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [15]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))

# Show the resulting DataFrame
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11234247|             1|
|   38676202|             1|
|   51245251|             1|
|   52523043|             1|
|   45717823|            13|
|   10828210|             6|
|   12854219|             1|
|   51715942|             1|
|   11477755|             2|
|   20024088|             3|
|    1486564|             1|
|    8648812|             1|
|    1619706|             1|
|   25622063|             2|
|   16435476|             8|
|    8062860|             1|
|    5632722|             2|
|   44537075|             1|
|   21072459|             1|
|   15367495|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [16]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))
            
# Show the resulting DataFrame
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
|R15C7QEZT0LGZN|          4|            2|          2|   N|
|R361XSS37V0NCZ|          1|            0|          0|   N|
| ROTNLALUAJAUB|          3|            0|          0|   N|
|R2DYX7QU6BGOHR|          5|            

# Load

In [17]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cukfnw6sp9ae.ap-southeast-2.rds.amazonaws.com:5432/proj4_uwaG4"
config = {"user":"root", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [18]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='watches_review_id_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='watches_products', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='watches_customers', mode=mode, properties=config)

In [21]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='watches_vine_table', mode=mode, properties=config)